## Medallion Architecture
- https://www.databricks.com/glossary/medallion-architecture
- https://learn.microsoft.com/en-us/fabric/onelake/onelake-medallion-lakehouse-architecture
- ![](https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png)

## Silver
- Define/Draft requirements, examples:
  - Calories to Weight Ratio.
- Transform and store semi-curate data based on requirements.
- Data Quality Check, examples:
  1. Uniqueness
  1. Completeness
  1. Validity
  1. Timeliness
  1. Accuracy
  1. Consistency

## Requirements
1. Must have an Unique key for each Recipe. (Completeness)
1. Weight must be in float data type. (Validity)
1. Ratio of Calories to Total Weight. (New column, >1 indicates less calories per weight)
1. If drink name, fruit 1 and fruit 2 are the same, it is the same recipe. (Uniqueness)
1. Weight, Calories and Ratio must not be empty. (Completeness)
1. Refresh Timestamp must be recorded. (Timeliness)
1. Weight must be > 0. (Accuracy)
1. All weight must be in the same unit. (Consistency)

In [0]:
desc odp_hackathon25.bronze.all_drink

In [0]:
drop table if exists odp_hackathon25.silver.drink_calories;

In [0]:
create table if not exists odp_hackathon25.silver.drink_calories
(
  recipe_key BIGINT generated always as identity,
  drink_name	string,
  creation_date	date,
  fruit1	string,
  fruit1_weight	float,
  fruit2	string,
  fruit2_weight	float,
  calories	float,
  calories_weight_ratio float,
  load_ts	timestamp
)

In [0]:
merge into odp_hackathon25.silver.drink_calories t
using (
  select 
    drink_name,
    creation_date,
    fruit1,
    fruit1_weight,
    fruit2,
    fruit2_weight,
    calories,
    calories / (cast(fruit1_weight as float) + cast(fruit2_weight as float)) as c_w_ratio
  from odp_hackathon25.bronze.all_drink
) s
on t.drink_name = s.drink_name and t.fruit1 = s.fruit1 and t.fruit2 = s.fruit2
when matched then
  update set
    t.drink_name = s.drink_name,
    t.creation_date = s.creation_date,
    t.fruit1 = s.fruit1,
    t.fruit1_weight = s.fruit1_weight,
    t.fruit2 = s.fruit2,
    t.fruit2_weight = s.fruit2_weight,
    t.calories = s.calories,
    t.calories_weight_ratio = s.c_w_ratio,
    t.load_ts = current_timestamp()
when not matched then
  insert 
    (
      drink_name, 
      creation_date, 
      fruit1, 
      fruit1_weight, 
      fruit2, 
      fruit2_weight,
      calories, 
      calories_weight_ratio, 
      load_ts
    )
    values (
      s.drink_name, 
      s.creation_date, 
      s.fruit1, 
      s.fruit1_weight, 
      s.fruit2, 
      s.fruit2_weight, 
      s.calories, 
      s.c_w_ratio,
      current_timestamp()
    )

In [0]:
select 
  assert_true(calories is not null, "calories must not be null"),
  assert_true((fruit1_weight + fruit2_weight)>0.0, "fruit weight must be more than 0"),
  assert_true(calories_weight_ratio is not null, "ratio must not be null") 
from odp_hackathon25.silver.drink_calories

## End of Silver Demo